In [1]:
import numpy as np
import pandas as pd
import torch
import torch.optim as optim
import torch.nn as nn
import torchvision
from torchvision.transforms import v2

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
image_size = (128, 128)
batch = 128
epochs = 50

In [4]:
transform = v2.Compose([
    v2.Resize(image_size),
    v2.ToTensor(),
])

/usr/local/lib/python3.11/dist-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [5]:
dataset = torchvision.datasets.ImageFolder("/kaggle/input/real-and-fake-images/dataset/train", transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=128, shuffle=True)

In [6]:
class MainNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.ModuleList([
            # -> [b, 3, 128, 128]
            nn.Conv2d(3, 32, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.BatchNorm2d(32),
            # -> [b, 32, 64, 64]
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.BatchNorm2d(64),
            # -> [b, 64, 32, 32]
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.BatchNorm2d(128),
            # -> [b, 128, 16, 16]
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.BatchNorm2d(256),
            # -> [b, 256, 8, 8]
            nn.Flatten(),
            nn.Linear(256*8*8, 1024),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(1024),
            
            nn.Linear(1024, 1)
        ])

    def forward(self, x):
        for layers in self.layers:
            x = layers(x)
        return x

In [7]:
class AuxNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.ModuleList([
            # -> [b, 3, 128, 128]
            nn.Conv2d(3, 32, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.BatchNorm2d(32),
            # -> [b, 32, 64, 64]
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.BatchNorm2d(64),
            # -> [b, 64, 32, 32]
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.BatchNorm2d(128),
            # -> [b, 128, 16, 16]
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.BatchNorm2d(256),
            # -> [b, 256, 8, 8]
            nn.Flatten(),
            nn.Linear(256*8*8, 1024),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(1024),
            
            nn.Linear(1024, 1)
        ])

    def forward(self, x):
        for layers in self.layers:
            x = layers(x)
        return x

In [8]:
mainNet = MainNet().to(device)
auxNet = AuxNet().to(device)

In [9]:
def build_route(main_layers, aux_layers, route_type):
    seq = []
    if route in (1, 2):
        for i, (main_l, aux_l) in enumerate(zip(main_layers, aux_layers)):
            if route_type == 1:
                if i % 2 == 0:
                    seq.append(aux_l)
                else:
                    seq.append(main_l)
    
            if route_type == 2:
                if i % 2 == 0:
                    seq.append(main_l)
                else:
                    seq.append(aux_l)
    else:
        seq = main_layers

    return nn.Sequential(*seq), seq

In [10]:
routes = [1, 2, 3]

all_params = list(mainNet.parameters()) + list(auxNet.parameters())

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(all_params, lr=1e-3)

In [11]:
for epoch in range(1, epochs+1):
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)
        Ls = {}
        for route in routes:
            
            submodel, layers = build_route(list(mainNet.layers), list(auxNet.layers), route)

            for p in all_params:
                p.requires_grad = False
            for l in layers:
                for p in l.parameters():
                    p.requires_grad = True

            out = submodel(images)
            loss = criterion(out.squeeze(1), labels.float())

            Ls[f'L{route}'] = loss

        L1, L2, L3 = Ls["L1"], Ls["L2"], Ls["L3"]
        
        alpha = 2 * L3.detach() / (L1.detach() + L2.detach() + 1e-8)
        loss_total = alpha * (L1 + L2) + L3
        
        optimizer.zero_grad()
        loss_total.backward()
        optimizer.step()

    print(f"[{epoch}/{epochs}] Loss: {loss_total}")

[1/50] Loss: 0.42758119106292725
[2/50] Loss: 0.33212199807167053
[3/50] Loss: 0.30787816643714905
[4/50] Loss: 0.20179173350334167
[5/50] Loss: 0.1232912540435791
[6/50] Loss: 0.18464219570159912
[7/50] Loss: 0.04163185507059097
[8/50] Loss: 0.18327262997627258
[9/50] Loss: 0.3189839720726013
[10/50] Loss: 0.04070073366165161
[11/50] Loss: 0.103192038834095
[12/50] Loss: 0.12198452651500702
[13/50] Loss: 0.14488382637500763
[14/50] Loss: 0.02706749737262726
[15/50] Loss: 0.05620477721095085
[16/50] Loss: 0.2506568133831024
[17/50] Loss: 0.023909512907266617
[18/50] Loss: 0.08002138137817383
[19/50] Loss: 4.4041324144927785e-05
[20/50] Loss: 0.039637889713048935
[21/50] Loss: 0.14927434921264648
[22/50] Loss: 0.057882316410541534
[23/50] Loss: 0.027636390179395676
[24/50] Loss: 0.12820006906986237
[25/50] Loss: 0.02788466215133667
[26/50] Loss: 0.000759470509365201
[27/50] Loss: 0.007667736150324345
[28/50] Loss: 0.030888449400663376
[29/50] Loss: 0.030532874166965485
[30/50] Loss: 0.0

In [12]:
val_dataset = torchvision.datasets.ImageFolder("/kaggle/input/real-and-fake-images/dataset/validation", transform=transform)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=128, shuffle=True)

In [13]:
mainNet.eval()
auxNet.eval()
val_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device).float()

        # Resultado do último route (por exemplo, route 3)
        submodel, _ = build_route(list(mainNet.layers), list(auxNet.layers), route_type=3)
        outputs = submodel(images).squeeze(1)
        
        loss = criterion(outputs, labels)
        val_loss += loss.item()

        preds = torch.sigmoid(outputs) >= 0.5
        correct += (preds == labels.bool()).sum().item()
        total += labels.size(0)

avg_val_loss = val_loss / len(val_loader)
val_accuracy = correct / total * 100

print(f"Val Loss: {avg_val_loss:.4f} | Val Acc: {val_accuracy:.2f}%")

mainNet.train()
auxNet.train()

Val Loss: 3.0224 | Val Acc: 76.97%


AuxNet(
  (layers): ModuleList(
    (0): Conv2d(3, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2)
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (4): LeakyReLU(negative_slope=0.2)
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (7): LeakyReLU(negative_slope=0.2)
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (10): LeakyReLU(negative_slope=0.2)
    (11): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): Flatten(start_dim=1, end_dim=-1)
    (13): Linear(in_features=16384, out_features=1024, bias=True)
    (14): LeakyReLU(negative_slope=0.2)
 

In [14]:
torch.save(mainNet.state_dict(), "main_net.pth")

In [15]:
torch.save(auxNet.state_dict(), "main_net.pth")